In [49]:
import cv2
import random
import mediapipe as mp
import pyautogui
import util
from pynput.mouse import Button,Controller
mouse = Controller()

In [50]:
screen_width, screen_height = pyautogui.size()

mpHands = mp.solutions.hands
hands = mpHands.Hands(
    static_image_mode = False,
    model_complexity = 1,
    min_detection_confidence = 0.7,
    min_tracking_confidence = 0.7,
    max_num_hands = 1
)


In [51]:
def find_finger_tip(processed):
    if processed.multi_hand_landmarks:
        hand_landMarks = processed.multi_hand_landmarks[0]
        return hand_landMarks.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP]
    return None

In [52]:
def move_mouse(index_finger_tip):
    if index_finger_tip is not None:
        x = int(index_finger_tip.x * screen_width)
        y = int(index_finger_tip.y * screen_height)
        pyautogui.moveTo(x,y)
        

In [53]:
def is_left_click(landMarks_list,thumb_index_dist):
    return (
            util.get_angle(landMarks_list[5],landMarks_list[6],landMarks_list[8])<50 and
            util.get_angle(landMarks_list[9],landMarks_list[10],landMarks_list[12])>90 and 
            thumb_index_dist > 50
     )
    
def is_right_click(landMarks_list,thumb_index_dist):
    return (
            util.get_angle(landMarks_list[9],landMarks_list[10],landMarks_list[12])<50 and
            util.get_angle(landMarks_list[5],landMarks_list[6],landMarks_list[8])>90 and 
            thumb_index_dist > 50
     )

def is_double_click(landMarks_list,thumb_index_dist):
    return (
            util.get_angle(landMarks_list[9],landMarks_list[10],landMarks_list[12])<50 and
            util.get_angle(landMarks_list[5],landMarks_list[6],landMarks_list[8])>50 and 
            thumb_index_dist > 50
     )
def is_screenshot(landMarks_list,thumb_index_dist):
    return (
            util.get_angle(landMarks_list[5],landMarks_list[6],landMarks_list[8])<50 and
            util.get_angle(landMarks_list[9],landMarks_list[10],landMarks_list[12])<50 and 
            thumb_index_dist < 50
     )

In [54]:
def detect_gestures(frame,landMarks_list,processed):
    if len(landMarks_list)>=21:
        index_finger_tip = find_finger_tip(processed)
        thumb_fing_dist = util.get_distance([landMarks_list[4],landMarks_list[5]])
        if thumb_fing_dist < 50 and util.get_angle(landMarks_list[5],landMarks_list[6],landMarks_list[8])>90:
            move_mouse(index_finger_tip)

        #Left Click 
        elif is_left_click(landMarks_list,thumb_fing_dist):
            mouse.press(Button.left)
            mouse.release(Button.left)
            cv2.putText(frame,"Left Click",(50, 50), cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)

        #RIGHT Click 
        elif is_right_click(landMarks_list,thumb_fing_dist):
            mouse.press(Button.right)
            mouse.release(Button.right)
            cv2.putText(frame,"Right Click",(50, 50), cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255), 2)

        #DOUBLE CLICK 
        elif is_double_click(landMarks_list,thumb_fing_dist):
            pyautogui.doubleClick()
            cv2.putText(frame,"Double Click",(50, 50), cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,0), 2)

        #SCREENSHOT 
        elif is_screenshot(landMarks_list,thumb_fing_dist):
            im1 = pyautogui.screenshot()
            label = random.randint(1,1000)
            im1.save(f'my_screenshot_{label}.png')
            cv2.putText(frame,"Screenshot Taken",(50,50),cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        

In [ ]:
def main():
    cap = cv2.VideoCapture(0)
    draw = mp.solutions.drawing_utils
    try:
        while cap.isOpened():
            ret,frame = cap.read()

            if not ret:
                break

            
            frame = cv2.flip(frame,1) 
            frameRGB = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
            processed = hands.process(frameRGB)

            landMarks_list = []
            if processed.multi_hand_landmarks:
                hand_landMarks = processed.multi_hand_landmarks[0]
                draw.draw_landmarks(frame, hand_landMarks, mpHands.HAND_CONNECTIONS)

                for l in hand_landMarks.landmark:
                    landMarks_list.append((l.x,l.y))
                
            detect_gestures(frame,landMarks_list,processed)
            
            cv2.imshow('FRAME', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break 
                
    finally:
        cap.release()
        cv2.destroyAllWindows() 

if __name__ == '__main__':
    main()